In [4]:
import nltk
import pandas as pd
import re
regex = re.compile('[^A-Za-zÀ-ÿ]')
import seaborn as sns
import numpy as np
import itertools
from tqdm import tqdm
from functools import reduce
import matplotlib.pyplot as plt
from collections import Counter
#from sklearn import svm

import itertools
import collections

import pickle

import matplotlib.pylab as plt

In [5]:
DIM_EMB = 20

In [6]:
COOC_PATH = './Results/cooc.pkl'
VOC_PATH = './Results/vocab.pkl'
CUT_VOC_PATH = './Results/vocab_cut.txt'
EMBEDDINGS_PATH = './Results/embeddings.npy'
DATA_TRAIN_POS_PATH = './Datasets/twitter-datasets/train_pos.txt' 
DATA_TRAIN_NEG_PATH = './Datasets/twitter-datasets/train_neg.txt' 
DATA_TEST_PP_PATH = './Results/res_test.txt' 

In [7]:
RESULT_POS_PATH = './Results/resPtrain_nd.txt'
RESULT_NEG_PATH = './Results/resNtrain_nd.txt'

In [8]:
with open(COOC_PATH, 'rb') as f:
    cooc_matrix = pickle.load(f)
    
with open(VOC_PATH, 'rb') as f:
    vocabulary = pickle.load(f)
    
embeddings = np.load(EMBEDDINGS_PATH)

# load the data files = list with each line being a tweet
pos_data =open(DATA_TRAIN_POS_PATH, "r").read().splitlines()
neg_data =open(DATA_TRAIN_NEG_PATH, "r").read().splitlines()
cut_vocabulary =open(CUT_VOC_PATH, "r").read().splitlines()
result_pos =open(RESULT_POS_PATH, "r").read().splitlines()
result_neg =open(RESULT_NEG_PATH, "r").read().splitlines()

test_data = open(DATA_TEST_PP_PATH, "r").read().splitlines()

In [9]:
test = [x.strip() for x in test_data[0].split(',')]

In [98]:
test

["['sea doo pro sea scooter sport portable sea doo seascootersave air stay long water'",
 "'shuck well work week come cheer put battery calculator'",
 "'stay away bug that baby'",
 "'madam lol perfectly fine contagious anymore mao'",
 "'whenever fall asleep watch always wake headache'",
 "'need get rid thing scare lol need car either need driver'",
 "'whatever terrible mood'",
 "'yes thanks jordan love gon call later'",
 "'friend text check last night'",
 "'follow back please unity tour come europe sweden'",
 "'watch dumb ass get lock today happy'",
 "'obsess phase kill best album ever love yew royce'",
 "'robert niro gay name like lewy would understand hahahahha sherlock'",
 "'canada grade since grade suck'",
 "'please say denmark would amazing live'",
 "'finally home'",
 "'custom picture frame poster frame wide complete gold frame fgd frame manufacture'",
 "'new follower mention followback boo'",
 "'yep look like best team stay proper form see bit'",
 "'nhs batsman suspension critici

In [99]:
embeddings.shape

(13191, 20)

In [100]:
pos = [x.strip() for x in result_pos[0].split(',')]
neg = [x.strip() for x in result_neg[0].split(',')]

In [101]:
pos

["['dunno justin read mention justin god know hope follow believe'",
 "'logic dumb even crop name photo tsk'",
 "'put casper box look battle bitch'",
 "'thanks sir trip lil mama keep doin'",
 "'visit brother best birthday gift every'",
 "'yay life complete tweet casebook let know please'",
 "'next album title feel rollercoaster life song concept life become famous follow plc'",
 "'work hard hardly work hardness future coworkers'",
 "'saw reply bit'",
 "'belong'",
 "'cheer national'",
 "'send invitation shop line find everything need without leave home'",
 "'wake fiona church'",
 "'agree day leave tho'",
 "'monet katemelo'",
 "'like damn lexis get lot say twitter lol'",
 "'grateful today dream fulfil heart full first complete track arrive back new york yes lord'",
 "'home affair shall later'",
 "'barra bout beat real madrid saturday doe'",
 "'lot part asia especially rat live country live grain suppose quite tasty'",
 "'even sleep shut cho ole back sleep look'",
 "'world best dad'",
 "'

In [8]:
# get vocab equivalence to tweet words

idx_pos_tweets = [[vocabulary.get((regex.sub(' ', ' '.join(regex.sub(' ', t).split()))), -1) for t in line.strip().split()] for line in pos]
idx_pos_tweets = [[t for t in tokens if t>=0] for tokens in idx_pos_tweets]

idx_neg_tweets = [[vocabulary.get((regex.sub(' ', ' '.join(regex.sub(' ', t).split()))), -1) for t in line.strip().split()] for line in pos]
idx_neg_tweets = [[t for t in tokens if t>=0] for tokens in idx_neg_tweets]

# get dense vector equivalence to tweet words
pos_tweets_word_vector = [[embeddings[wd2voc][:] for wd2voc in tweet_words] for tweet_words in idx_pos_tweets]

neg_tweets_word_vector = [[embeddings[wd2voc][:] for wd2voc in tweet_words] for tweet_words in idx_neg_tweets]

# get mean word vector of each tweet
pos_tweets_mean_vector = [np.mean(wordvectors,axis=0) for wordvectors in pos_tweets_word_vector]

neg_tweets_mean_vector = [np.mean(wordvectors,axis=0) for wordvectors in neg_tweets_word_vector]

/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
label_pos = [1] * len(pos)
pos_df = pd.DataFrame(list(zip(label_pos, pos, idx_pos_tweets,pos_tweets_word_vector,pos_tweets_mean_vector)),\
                      columns=["Sentiment","Tweet","Token_idx","Words_Vectors","Mean_Word_Vector"]) #create a df
del label_pos
pos_df.head()

,Sentiment,Tweet,Token_idx,Words_Vectors,Mean_Word_Vector
0,1,['dunno justin read mention justin god know ho...,"[1312, 439, 206, 459, 439, 140, 4, 45, 9, 209]","[[-0.14710372696628293, 1.086682084363625, -0....","[-0.35072209557061357, 0.7329244231493826, -0...."
1,1,'logic dumb even crop name photo tsk',"[2094, 1194, 68, 4664, 173, 309, 4455]","[[-1.556495316687639, 1.9481823643797989, -0.8...","[-0.39852621887554446, 0.2221294616363131, 0.0..."
2,1,'put casper box look battle bitch',"[156, 7907, 308, 25, 1357, 196]","[[-0.6393250135775723, -0.24818839004606633, -...","[0.03351917175659059, 0.09320186637661715, 0.0..."
3,1,'thanks sir trip lil mama keep doin',"[26, 1109, 525, 420, 856, 98, 1453]","[[-0.5080396613782544, 0.37463485113504796, -0...","[-0.2263552822782026, 0.5498975401627835, 0.19..."
4,1,'visit brother best birthday gift every',"[487, 271, 43, 91, 725, 172]","[[-0.47369600816379476, -0.30626771725887386, ...","[-0.2748481111946549, 0.14258626686447956, 0.0..."


In [10]:
label_neg = [-1] * len(neg)
neg_df = pd.DataFrame(list(zip(label_neg,neg, idx_neg_tweets,neg_tweets_word_vector,neg_tweets_mean_vector)),\
                      columns=["Sentiment","Tweet","Token_idx","Words_Vectors","Mean_Word_Vector"]) #create a df
del label_neg
neg_df.head(1000)

,Sentiment,Tweet,Token_idx,Words_Vectors,Mean_Word_Vector
0,-1,['vince tresorpack difficulty object disassemb...,"[1312, 439, 206, 459, 439, 140, 4, 45, 9, 209]","[[-0.14710372696628293, 1.086682084363625, -0....","[-0.35072209557061357, 0.7329244231493826, -0...."
1,-1,'glad dot take tomorrow thankful start',"[2094, 1194, 68, 4664, 173, 309, 4455]","[[-1.556495316687639, 1.9481823643797989, -0.8...","[-0.39852621887554446, 0.2221294616363131, 0.0..."
2,-1,'celtic regular season fuck play playoff',"[156, 7907, 308, 25, 1357, 196]","[[-0.6393250135775723, -0.24818839004606633, -...","[0.03351917175659059, 0.09320186637661715, 0.0..."
3,-1,'could actually kill girl sorry',"[26, 1109, 525, 420, 856, 98, 1453]","[[-0.5080396613782544, 0.37463485113504796, -0...","[-0.2263552822782026, 0.5498975401627835, 0.19..."
4,-1,'find hard believe afraid',"[487, 271, 43, 91, 725, 172]","[[-0.47369600816379476, -0.30626771725887386, ...","[-0.2748481111946549, 0.14258626686447956, 0.0..."
...,...,...,...,...,...
995,-1,'hope expensive',"[10, 1082, 540, 229, 9]","[[-0.4501262187043856, 0.39455591364676856, -0...","[-0.5294881894522154, 0.3331186413435129, 0.31..."
996,-1,'veto lift bring mcdonalds',"[193, 1995, 1785, 2010, 179, 3018, 331]","[[0.4379044107300885, 1.5083934849720537, -0.7...","[-0.0889891269310521, 0.4063476666019291, 0.08..."
997,-1,'want grow big fat like badly',"[605, 93, 216, 6, 896, 3268, 50]","[[-0.3091583487072667, 1.167363582424688, -0.4...","[-0.44712345374967594, 0.6397136345706483, -0...."
998,-1,'wake soo sad',[33],"[[-0.5223331548199615, 0.5166696367743184, -0....","[-0.5223331548199615, 0.5166696367743184, -0.4..."


---
Ligne suivante regroupe pos et neg

In [11]:
full_df = pd.concat([pos_df,neg_df],ignore_index=True) #regroup the dfs, ignore index in order to get new ones (->no duplicate)

full_df = full_df.sample(frac=1) #shuffles the rows

In [12]:
full_df["Mean_Word_Vector"].isna().sum()

208

In [13]:
full_df.shape

(171972, 5)

In [14]:
full_df

,Sentiment,Tweet,Token_idx,Words_Vectors,Mean_Word_Vector
103639,-1,'want cry little ill tomorrow',"[7793, 851, 2998, 140, 486, 7748, 7793]","[[-0.6985876958373647, -1.5228579316073594, -1...","[-0.497975768460799, -0.30284411574244613, -1...."
73743,1,'love mum make bacon sandwich morning everythi...,"[1, 450, 8, 1983, 1719, 75, 149, 389, 114, 270...","[[-0.4782546316313907, 0.39935830336821215, -0...","[-0.6119139621077291, 0.3808678152496684, -0.0..."
102224,-1,'gel nail make everything hard',"[1265, 122, 31]","[[0.31942221128879705, 1.6903793944000731, -0....","[-0.19436230840585852, 0.6892238508493594, -0...."
37886,1,'work way',"[27, 56]","[[-0.4499319801133071, 0.3353113695696592, -0....","[-0.43765819647448984, 0.2731880708094367, -0...."
101247,-1,'give jacket cold aaaaa loquelomutso',[3106],"[[1.069085729310904, -0.7671984806679858, -0.8...","[1.069085729310904, -0.7671984806679858, -0.80..."
...,...,...,...,...,...
31475,1,'read mine tomorrow freak tired enjoy much wai...,"[206, 220, 36, 628, 227, 235, 29, 48, 206]","[[-0.014431184738647667, 0.37269162048978277, ...","[-0.26470829191282286, 0.10798328372436851, -0..."
19675,1,'still december',"[37, 3683]","[[-0.4602956871131378, 0.4270518036265086, -0....","[0.1037925742516134, 1.061909632487603, -0.477..."
41504,1,'imagine sort hand gesture learn italian pract...,"[746, 867, 295, 362, 1645, 4370, 62, 295, 562]","[[-0.43658123705005747, 0.8769457361918845, -0...","[-0.20071754949616033, 0.21328737877939108, 0...."
148194,-1,'tale body thief vampire chronicle mass market...,"[7, 19, 188, 3, 3, 1936]","[[-0.5275402626308756, 0.43521171009270515, -0...","[-0.2864346381617253, 0.3326479125995642, -0.3..."


In [15]:
tab = full_df.dropna()

In [16]:
tab.shape[0]

171764

In [17]:
X = tab['Mean_Word_Vector'].to_numpy()

In [18]:
import numpy

In [19]:
X

array([array([-0.49797577, -0.30284412, -1.03888662,  0.35407835,  0.52622889,
       -0.5505717 ,  0.39591095, -0.17754479,  0.55846175, -0.13133973,
        0.20809187,  0.38206573,  0.41377463, -0.25216436,  0.1070671 ,
        0.08776263,  0.45870062, -0.46427218, -0.52760499, -0.08624541]),
       array([-0.61191396,  0.38086782, -0.01446227,  0.24157714, -0.44264714,
        0.25873044,  0.24498193, -0.40734023, -0.05342761, -0.24861718,
       -0.02824705,  0.07146502,  0.60523615,  0.18353647, -0.39337313,
       -0.21613943,  0.19013019,  0.11205012,  0.28321444, -0.05871216]),
       array([-0.19436231,  0.68922385, -0.45696355,  0.19340391, -0.60902677,
        0.41656931, -0.29166539,  0.30720966, -0.25007545, -0.08417561,
        0.08739296, -0.41601523,  0.71429202,  0.37301231,  0.42128255,
        0.21938701,  0.15426109,  0.15891003,  0.11906925, -0.44900311]),
       ...,
       array([-0.20071755,  0.21328738,  0.08802651,  0.00326636, -0.72061896,
        0.13573706

In [20]:
X = numpy.concatenate(X, axis=0).reshape((tab.shape[0],DIM_EMB))

In [21]:
X

array([[-0.49797577, -0.30284412, -1.03888662, ..., -0.46427218,
        -0.52760499, -0.08624541],
       [-0.61191396,  0.38086782, -0.01446227, ...,  0.11205012,
         0.28321444, -0.05871216],
       [-0.19436231,  0.68922385, -0.45696355, ...,  0.15891003,
         0.11906925, -0.44900311],
       ...,
       [-0.20071755,  0.21328738,  0.08802651, ...,  0.19594965,
         0.31218171,  0.15574838],
       [-0.28643464,  0.33264791, -0.33396704, ..., -0.14949501,
         0.22983457, -0.56555351],
       [-0.60655708,  0.15169334, -0.25038021, ...,  0.43166156,
         0.23144664, -0.06633554]])

In [22]:
X.shape

(171764, 20)

In [23]:
y = tab['Sentiment'].to_numpy()
y
y.shape

(171764,)

In [24]:
#y = np.array([y]).T
#np.shape(y)

In [25]:
from SVM import *
from CV import *

In [26]:
w=[1,2,3,4,5,5,5,5,5,5,5,5,5,5,5,5,55,5,1,2]
np.shape(w)

(20,)

In [27]:
a = X@w
a

array([27.24178157, 10.12673607, 13.52772048, ..., 13.38813553,
       20.7245141 , 10.53731299])

In [28]:
w = sgd_for_svm_demo(y, X)

hingle_loss
primal obj
iteration=0, cost=19569302396.90662
hingle_loss
primal obj
iteration=10000, cost=506754220.6407911
hingle_loss
primal obj
iteration=20000, cost=308569319.9742599
hingle_loss
primal obj
iteration=30000, cost=195476955.57616925
hingle_loss
primal obj
iteration=40000, cost=119989077.5179448
hingle_loss
primal obj
iteration=50000, cost=61474487.744578525
hingle_loss
primal obj
iteration=60000, cost=14337649.38948698
hingle_loss
primal obj
iteration=70000, cost=415501.74028428167
hingle_loss
primal obj
iteration=80000, cost=416124.2639569292
hingle_loss
primal obj
iteration=90000, cost=353603.73640561383
pred
(171764, 20)
(20,)
(171764,)
pred done
compute acc
acc
pred
(171764, 20)
(20,)
(171764,)
pred done
compute f1
f1
training accuracy = 0.5 
 training F1 score = 0.5866486980796072


In [29]:
a = [1,2,3,4]
y2= [1,2,6,7]

In [30]:
np.sum(a == y2)

0

In [31]:
cross_validation_demo(y, X)

hingle_loss
primal obj
iteration=0, cost=12318645356.153587
hingle_loss
primal obj
iteration=10000, cost=3885041785.2326293
hingle_loss
primal obj
iteration=20000, cost=3632097439.010739
hingle_loss
primal obj
iteration=30000, cost=3484123354.8346186
hingle_loss
primal obj
iteration=40000, cost=3377928800.0278635
hingle_loss
primal obj
iteration=50000, cost=3299400123.7177644
hingle_loss
primal obj
iteration=60000, cost=3232849666.6994085
hingle_loss
primal obj
iteration=70000, cost=3176582995.8471146
hingle_loss
primal obj
iteration=80000, cost=3129235379.6580577
hingle_loss
primal obj
iteration=90000, cost=3086995022.856129
pred
(171764, 20)
(20,)
(171764,)
pred done
compute acc
acc
pred
(171764, 20)
(20,)
(171764,)
pred done
compute f1
f1
training accuracy = 0.5 
 training F1 score = 0.5038533085304279
hingle_loss
primal obj
hingle_loss
primal obj
hingle_loss
primal obj
iteration=0, cost=33791879527.52793
hingle_loss
primal obj
iteration=10000, cost=442557210.58911884
hingle_loss
pr

KeyboardInterrupt: 

In [ ]:
coordinate_descent_for_svm_demo(y, X)

In [47]:
X

array([[-0.49797577, -0.30284412, -1.03888662, ..., -0.46427218,
        -0.52760499, -0.08624541],
       [-0.61191396,  0.38086782, -0.01446227, ...,  0.11205012,
         0.28321444, -0.05871216],
       [-0.19436231,  0.68922385, -0.45696355, ...,  0.15891003,
         0.11906925, -0.44900311],
       ...,
       [-0.20071755,  0.21328738,  0.08802651, ...,  0.19594965,
         0.31218171,  0.15574838],
       [-0.28643464,  0.33264791, -0.33396704, ..., -0.14949501,
         0.22983457, -0.56555351],
       [-0.60655708,  0.15169334, -0.25038021, ...,  0.43166156,
         0.23144664, -0.06633554]])

In [48]:
def standardize(x):
    cent_data = x - np.mean(x, axis=0)
    std_data = cent_data / np.std(cent_data, axis=0)
    return std_data, np.mean(x, axis=0), np.std(cent_data, axis=0)


In [49]:
X_, _, _ = standardize(X)
X_

array([[-0.80865035, -1.92451123, -3.05782205, ..., -2.25968874,
        -2.509544  ,  0.48941603],
       [-1.19306097,  0.52149917,  0.45387283, ..., -0.27108651,
         0.32030004,  0.58315218],
       [ 0.2156966 ,  1.62465685, -1.06300781, ..., -0.10939623,
        -0.25258373, -0.74558169],
       ...,
       [ 0.19425495, -0.07802743,  0.80520118, ...,  0.01840912,
         0.42139882,  1.31327665],
       [-0.09494183,  0.34899028, -0.64137959, ..., -1.17354897,
         0.13399873, -1.14237403],
       [-1.17498764, -0.29838288, -0.35484655, ...,  0.83173394,
         0.13962503,  0.5571986 ]])

In [ ]:
type(y[0])

In [32]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification

clf = LinearSVC(penalty='l2', loss='hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', 
          fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)
clf.fit(X, y)



LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
          penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [33]:
print(clf.coef_)

[[ 1.33851263e-14 -3.88022947e-14  8.78117024e-15  3.60822483e-16
   5.23192600e-15  2.72143419e-14 -4.54497551e-15 -5.67601521e-15
   2.05668815e-14  8.70831185e-15 -1.04326270e-14  6.66133815e-15
  -1.04290491e-14 -2.04836148e-14 -3.47985529e-15 -1.26643487e-14
  -9.31893451e-15  1.55014890e-14 -4.64905892e-15  1.59941504e-14]]


In [34]:
print(clf.classes_)

[-1  1]


In [35]:
print(compute_F1score(y, X, np.squeeze(clf.coef_)))

pred
(171764, 20)
(20,)
(171764,)
pred done
compute f1
f1
0.2047962962962963


In [36]:
print(calculate_accuracy(y, X, np.squeeze(clf.coef_)))

pred
(171764, 20)
(20,)
(171764,)
pred done
compute acc
acc
0.5


In [37]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [39]:
X_train.shape, y_train.shape

((120234, 20), (120234,))

In [40]:
X_test.shape, y_test.shape

((51530, 20), (51530,))

In [41]:
from sklearn import svm
clf = LinearSVC(penalty='l2', loss='hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000).fit(X_train, y_train)
clf.score(X_test, y_test)

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.49607995342518924

In [57]:
from sklearn.preprocessing import normalize
X_n = normalize(X)
X_n

array([[-0.25556403, -0.15542135, -0.5331626 , ..., -0.23826716,
        -0.27076992, -0.04426164],
       [-0.4473233 ,  0.27842321, -0.01057225, ...,  0.08191124,
         0.20703633, -0.04291995],
       [-0.11466778,  0.40662085, -0.26959442, ...,  0.09375202,
         0.07024719, -0.26489801],
       ...,
       [-0.16596403,  0.17635744,  0.07278504, ...,  0.16202168,
         0.25812857,  0.12878111],
       [-0.20461986,  0.23763316, -0.2385755 , ..., -0.10679452,
         0.16418656, -0.40401356],
       [-0.39083842,  0.09774445, -0.16133388, ...,  0.27814352,
         0.14913393, -0.04274367]])

In [66]:
from sklearn.model_selection import cross_validate
clf = LinearSVC(penalty='l2', loss='hinge', dual=True, tol=0.0001, C=0.01, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=100000)
cv_results = cross_validate(clf, X_, y, cv=5)
print(cv_results)

{'fit_time': array([0.4623909 , 0.45133424, 0.41323113, 1.5699451 , 1.29792619]), 'score_time': array([0.00625896, 0.00539589, 0.00361085, 0.00324082, 0.00305891]), 'test_score': array([0.49190778, 0.49414915, 0.49339194, 0.48931649, 0.4960701 ])}
